In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np 
import scipy as sp 
import sklearn
import random 
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from sklearn import preprocessing, model_selection
from keras.models import Sequential 
from keras.layers import Dense 
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from keras.layers import Embedding, LSTM, SpatialDropout1D
data = pd.read_csv('antversion.csv')
#data = shuffle(data)


In [10]:
#data = data.drop(['name', 'version', 'name'], axis =1)
X = data.drop(['bug'], axis = 1)
X = np.array(X)
Y = data['bug']
Y.shape

(1692,)

In [11]:
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
data[cols_to_norm] = MinMaxScaler().fit_transform(data[cols_to_norm])
print(data.head())

        wmc       dit       noc       cbo       rfc      lcom        ca  \
0  0.091667  0.500000  0.019608  0.028056  0.145833  0.004334  0.004016   
1  0.116667  0.000000  0.009804  0.016032  0.111111  0.007322  0.008032   
2  0.025000  0.166667  0.000000  0.002004  0.031250  0.000000  0.000000   
3  0.100000  0.333333  0.000000  0.024048  0.128472  0.004782  0.000000   
4  0.050000  0.333333  0.000000  0.008016  0.072917  0.000149  0.000000   

         ce       npm     lcom3  ...  dam       moa       mfa       cam   ic  \
0  0.324324  0.048544  0.362500  ...  1.0  0.090909  0.885057  0.232323  0.6   
1  0.108108  0.116505  0.417582  ...  1.0  0.000000  0.000000  0.307692  0.0   
2  0.027027  0.009709  0.000000  ...  1.0  0.090909  0.714286  0.666667  0.2   
3  0.324324  0.116505  0.429293  ...  1.0  0.090909  0.770833  0.458333  0.0   
4  0.108108  0.058252  0.350000  ...  1.0  0.000000  0.880952  0.416667  0.4   

        cbm       amc    max_cc    avg_cc  bug  
0  0.210526  0.0168

In [12]:
ros = RandomOverSampler(random_state=0)
X_resampled, Y_resampled = ros.fit_resample(X, Y)

C:\Users\SUSHANT\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [15]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(X_resampled, Y_resampled,test_size = 0.2, random_state = 0)
smt = SMOTE()
train_x, train_y = smt.fit_sample(train_x, train_y)
train_x = np.reshape(train_x, (train_x.shape[0], 1, train_x.shape[1]))
test_x = np.reshape(test_x, (test_x.shape[0], 1, test_x.shape[1]))
input_dim = len(data.columns) - 1
#train_x, train_y = smt.fit_sample(train_x, train_y)
model = Sequential()
#model.add(Embedding(1024, 1, input_length=20))
model.add(LSTM(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
#model.add(LSTM(512, dropout = 0.3, return_sequences=True))
#model.add(LSTM(256, dropout = 0.4, return_sequences=True))
model.add(LSTM(80, dropout = 0.2, return_sequences=True))
model.add(LSTM(60, dropout = 0.2, return_sequences=False))
#model.add(LSTM(16, input_shape=(1,20), dropout = 0.67))
#model.add(Dense(1, activation='softmax'))
#model.add(SpatialDropout1D(0.7))
#model.add(LSTM(64, dropout=0.7))
#model.add(LSTM(16, dropout=0.7, return_sequences=True))
#model.add(LSTM(8, dropout=0.7))
#model.add(Dense(10, input_dim = input_dim , activation = 'relu'))
#model.add(Dense(10, activation = 'relu'))
#model.add(Dense(8, activation = 'relu'))
#model.add(Dense(8, activation = 'relu'))
#model.add(Dense(6, activation = 'relu'))
#model.add(Dense(32, activation = 'relu'))
model.add(Dense(11, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy','mse', 'mae', 'mape', 'cosine'] )
history = model.fit(train_x, train_y, validation_split = 0.3, epochs = 10, batch_size = 128)
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))
print("\n%s: %.2f%%" % (model.metrics_names[4], score[4]))
print("\n%s: %.2f%%" % (model.metrics_names[5], score[5]*100))
sklearn.metrics.matthews_corrcoef(train_y, test_y, sample_weight=None)
#print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()



C:\Users\SUSHANT\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\Users\SUSHANT\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\Users\SUSHANT\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\Users\SUSHANT\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(m

Train on 7671 samples, validate on 3289 samples
Epoch 1/10
7671/7671 [==============================] - 4s 465us/step - loss: 2.2833 - accuracy: 0.2314 - mse: 29.7360 - mae: 4.5362 - mape: 9113513.0000 - cosine: 2.8951 - val_loss: 1.9517 - val_accuracy: 0.4226 - val_mse: 29.1942 - val_mae: 4.5064 - val_mape: 9038479.0000 - val_cosine: 2.5827
Epoch 2/10
7671/7671 [==============================] - 1s 100us/step - loss: 1.8282 - accuracy: 0.3589 - mse: 29.7440 - mae: 4.5362 - mape: 9113514.0000 - cosine: 2.2463 - val_loss: 1.4922 - val_accuracy: 0.4792 - val_mse: 29.2030 - val_mae: 4.5064 - val_mape: 9038478.0000 - val_cosine: 2.0227
Epoch 3/10
7671/7671 [==============================] - 1s 104us/step - loss: 1.5814 - accuracy: 0.4224 - mse: 29.7511 - mae: 4.5362 - mape: 9113514.0000 - cosine: 1.9190 - val_loss: 1.2643 - val_accuracy: 0.4773 - val_mse: 29.2115 - val_mae: 4.5064 - val_mape: 9038478.0000 - val_cosine: 1.7658
Epoch 4/10
7671/7671 [==============================] - 1s 105us

ValueError: Found input variables with inconsistent numbers of samples: [10960, 2684]

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 100)            48400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 80)             57920     
_________________________________________________________________
lstm_3 (LSTM)                (None, 60)                33840     
_________________________________________________________________
dense_1 (Dense)              (None, 11)                671       
Total params: 140,831
Trainable params: 140,831
Non-trainable params: 0
_________________________________________________________________
